In [2]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/arcane/')

from src._utils import load_bertopic_model_from_hf

/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-pac

In [18]:
from sql.articles.ArticleAttributesSQL import ArticleAttributesSQL

In [4]:
import pandas as pd
import numpy as np
import json

In [6]:
df = pd.read_csv('all_article_attributes_WatermelonSapphireZipline.csv')

In [7]:
len(df)

10966

In [8]:
df.article_id.nunique()

10654

In [10]:
articles = {row.article_id: json.loads(row.attributes) for row in df.itertuples()}

In [11]:
len(articles)

10654

In [20]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [13]:
import time

In [16]:
from src.articles.ArticleAttributesService import ArticleAttributesService

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


In [28]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT DISTINCT article_id FROM llm_article_attributes'
    cursor.execute(sql)
    results = cursor.fetchall()
db_saved_results = [x[0] for x in results]

In [29]:
pending_articles = set(articles.keys()) - set(db_saved_results)

In [25]:
pending_article_attributes = {article_id: attributes for article_id, attributes in articles.items() if article_id in pending_articles}

In [27]:
completed = 0
start_time = time.time()
invalid_articles = []
for article_id, article_attributes in pending_article_attributes.items():
    try:
        ArticleAttributesService.save_llm_article_attributes(article_id=article_id, article_attributes=article_attributes)
        completed += 1
        if completed % 100 == 0:
            print(f'completed {completed} in {time.time() - start_time} seconds')
    except:
        invalid_articles.append(article_id)
        print(f'article_id {article_id} is invalid')

completed 100 in 22.978924989700317 seconds
completed 200 in 45.89204001426697 seconds
completed 300 in 69.36836218833923 seconds
completed 400 in 92.23325991630554 seconds
article_id 651dc736a662d76276b7b8ae is invalid
completed 500 in 114.56410908699036 seconds
completed 600 in 142.0421130657196 seconds
completed 700 in 170.21020317077637 seconds
completed 800 in 197.18556904792786 seconds
completed 900 in 220.73140597343445 seconds
completed 1000 in 272.73933815956116 seconds
completed 1100 in 301.4587731361389 seconds
article_id 651dca24a662d76276b7c0da is invalid
completed 1200 in 325.1629910469055 seconds
completed 1300 in 348.51268696784973 seconds
completed 1400 in 371.5058002471924 seconds
completed 1500 in 394.9644021987915 seconds
completed 1600 in 417.5018150806427 seconds
completed 1700 in 439.99999618530273 seconds
completed 1800 in 467.857568025589 seconds
article_id 651dcccfa662d76276b7c808 is invalid
completed 1900 in 492.623370885849 seconds
completed 2000 in 518.9174

In [70]:
articles['651dacd0701055199ecf9966']

{'analysis_of_article_sentiment': 'The article is neutral in nature, neither bullish nor bearish.',
 'analysis_of_article_type': 'The article provides factual information on a football match, as well as details about the teams and their past performances.',
 'analysis_of_article_validity_duration': 'The article pertains to a football match happening on Wednesday, October 4, hence its relevance duration is for one day.',
 'analysis_of_financial_or_business_news': "The article pertains to football, specifically UEFA Champions League, and doesn't cover financial or business news.",
 'analysis_of_popularity': 'The article is about a popular sport, football. It would likely appeal to football fans, making it moderately popular.',
 'analysis_of_relevant_for_india': 'This article is about a UEFA Champions League match between Manchester City and RB Leipzig, which is an international sporting event, relevant for football enthusiasts worldwide including Indians.',
 'article_sentiment': 'na',
 '

In [5]:
len('This article details private currencies their role and their usage throughout history. It also addresses global trends such as digital currencies. While private currencies have usage limitations many still exist. Some currencies have been backed by physical commodities to enhance their value. In terms of legality creating private currencies in the US is prohibited.')

367

In [64]:
from src.articles.ArticleAttributesService import ArticleAttributesService

In [65]:
ArticleAttributesService.save_llm_article_attributes?

Signature:
ArticleAttributesService.save_llm_article_attributes(
    article_id,
    article_attributes,
)
Docstring: <no docstring>
File:      ~/Desktop/ML/Recommendations/arcane/src/articles/ArticleAttributesService.py
Type:      function

In [52]:
def save_summary_json_details_to_mongo(article_id, article_attributes, finetune_id):
    url = f'http://insight-user-app-beta-env.eba-rnrpvmin.ap-south-1.elasticbeanstalk.com/cloud/articles/{article_id}/ai-generated-info'
    data = {'model': finetune_id, 'summary': {'value': article_attributes['final_summary'], 'additionalInfo': {}}, 'title': {'value': article_attributes['headline_suggestion'], 'additionalInfo': {}}}
    r = requests.put(url,json=data)
    return r

In [53]:
r = save_summary_json_details_to_mongo(article_id=random_row.article_id, article_attributes=json.loads(random_row.attributes), finetune_id=finetune_id)

In [54]:
r.status_code

200

In [55]:
r.text

'{"message":"Success"}'

In [27]:
json.loads(random_row.attributes)['final_summary']

"India's capital markets regulator SEBI is proposing to set up a Performance Validation Agency (PVA) to authenticate the performance claims of registered intermediaries, including investment advisers, research analysts, and portfolio managers. This measure is expected to enhance credibility in the intermediaries' services and potentially draw more clients. The agency would validate performance against specified parameters such as returns, risk, and volatility. SEBI seeks comments on this proposal till September 21."

In [29]:
json.loads(random_row.attributes)['headline_suggestion']

'SEBI to Set Up Performance Validation Agency to Authenticate Intermediaries'

In [21]:
random_row.attributes

'{"analysis_of_article_sentiment": "The article portrays a neutral sentiment as it presents a new regulation without expressing any positive or negative implications.", "analysis_of_article_type": "The article is primarily a factual piece providing information about a new proposed regulation by SEBI.", "analysis_of_article_validity_duration": "This news is about a regulatory change proposed by SEBI, which will affect the intermediaries in the market for a longer period.", "analysis_of_financial_or_business_news": "The article is about SEBI\'s proposal to set up a Performance Validation Agency (PVA) to validate intermediaries\' claims, which falls under finance and business news.", "analysis_of_popularity": "SEBI being a financial regulator in India, this news is likely to be of high interest to financial professionals and the general investing public.", "analysis_of_relevant_for_india": "The article is specifically about SEBI, the Indian securities market regulator, and its proposed re

In [22]:
ArticleAttributesSQL.save_article_attributes(article_id=random_row.article_id, article_attributes=json.loads(random_row.attributes), finetune_id='WatermelonSapphireZipline')

In [11]:
json.loads(random_row.attributes)

{'analysis_of_article_sentiment': 'The sentiment is bullish as the issuance of NCDs successfully raised Rs 200 crore.',
 'analysis_of_article_type': 'This is a factual article, presenting information about the successful bond issuance by Credit Saison India.',
 'analysis_of_article_validity_duration': 'This is a one-time event and a significant achievement for the company. It will stay relevant for a few days.',
 'analysis_of_financial_or_business_news': 'This article discusses the successful issuance of NCDs by Credit Saison India, a non-banking financial corporation.',
 'analysis_of_popularity': 'This news is likely to be moderately popular as it pertains to the successful bond issuance by a non-banking financial corporation, Credit Saison India.',
 'analysis_of_relevant_for_india': 'This is relevant for India as it discusses a significant financial event of a non-banking financial corporation based in India, Credit Saison India.',
 'article_sentiment': 'bull',
 'article_type': 'fact

In [140]:
results = {row.article_id: json.loads(row.attributes) for row in df.itertuples()}

In [142]:
df.sample(40)

,article_id,attributes
4756,651dd66ca662d76276b7e22b,"{""analysis_of_article_sentiment"": ""The sentime..."
3664,651dd1bfa662d76276b7d5e9,"{""analysis_of_article_sentiment"": ""The sentime..."
2816,651dce96a662d76276b7ccf9,"{""analysis_of_article_sentiment"": ""The sentime..."
6155,651ddc21a662d76276b7f0cb,"{""analysis_of_article_sentiment"": ""The sentime..."
913,651dc7aaa662d76276b7ba09,"{""analysis_of_article_sentiment"": ""The article..."
4680,651dd617a662d76276b7e160,"{""analysis_of_article_sentiment"": ""The article..."
5833,651ddae1a662d76276b7ed80,"{""analysis_of_article_sentiment"": ""The article..."
6407,651ddd41a662d76276b7f3c7,"{""analysis_of_article_sentiment"": ""The article..."
5909,651ddb16a662d76276b7ee09,"{""analysis_of_article_sentiment"": ""The article..."
2243,651dcce1a662d76276b7c83b,"{""analysis_of_article_sentiment"": ""The sentime..."


In [82]:
s = pd.DataFrame([k['financial_or_business_news'] for i, k in results.items() if 'financial_or_business_news' in k])

In [83]:
s.groupby(0)[0].count()

0
False    671
True     885
Name: 0, dtype: int64

In [144]:
results['651ddd41a662d76276b7f3c7']

{'analysis_of_article_sentiment': 'The article is neutral as it presents factual information without expressing a positive or negative sentiment.',
 'analysis_of_article_type': 'This article is factual, providing information about Vivek Ramaswamy and his political journey.',
 'analysis_of_article_validity_duration': 'This article is relevant until the next major event in the 2024 Presidential primaries, approximately a week.',
 'analysis_of_financial_or_business_news': "The article focuses on Vivek Ramaswamy, a businessman running for the Republican Party's nomination for the 2024 US Presidential elections.",
 'analysis_of_popularity': 'The news is of interest to those interested in US politics and business, and may have moderate popularity.',
 'analysis_of_relevant_for_india': "Though focused on an Indian-origin candidate in the US politics, the article holds minimal relevance for an Indian audience, as it doesn't directly pertain to India or Indian corporations.",
 'article_sentiment

In [ ]:
ArticleAttributesSQL.